In [1]:
import nltk
import re
from nltk.tokenize import word_tokenize,sent_tokenize
#wt = word_tokenize()
#st = sent_tokenize()

from nltk.stem import PorterStemmer
ps = PorterStemmer()

from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
lt = WordNetLemmatizer()


from nltk.corpus import stopwords
s_words = stopwords.words('english')
import string
pun = list(string.punctuation)
s_words += pun


In [2]:
#getting pos in useful language
from nltk.corpus import wordnet
def get_simple_pos (tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


In [3]:
def clean_text(words):
    output_words=[]
    for w in words:
        if w not in s_words:
            w = w.replace(r'\S+@\S+', ' ')  # remove email
            w = re.sub('[^a-z,A-Z]',' ',w)
            #pos = pos_tag(w)
            #clean_w = lt.lemmatize(w,pos =get_simple_pos(pos[0][1]))
            clean_w = ps.stem(w)
            clean_w = clean_w.strip()  # remove trailing leading spaces
            clean_w = clean_w.replace(r'\s+', '')  # remove whitespace
            output_words.append(clean_w)
    return output_words

In [4]:
def get_words(path):
    import glob
    all_cat = glob.glob(path + "/*")
    documents = []
    for cc in all_cat:
        n_path = cc
        all_doc = glob.glob(n_path +"/*")
        for cd in all_doc:
            f = '/'.join(cd.split('\\'))
            f1 = open(f, 'r').read().replace("\n"," ").lower()
            words = word_tokenize(f1)
            documents.append((clean_text(words),cc))
    import random
    random.shuffle(documents)
    return documents

In [5]:
path = r'C:\Users\YATISH\Desktop\NB_TEXT_ANALYSIS\20_newsgroups'
all_docs = get_words(path)
print(len(all_docs))

19997


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 50000)


In [7]:
categories = [all_docs[i][1] for i in range (len(all_docs))]
text_docs = [" ".join(all_docs[i][0]) for i in range (len(all_docs))]


In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(text_docs,categories)

In [9]:
X_train = cv.fit_transform(x_train)
X_test = cv.transform(x_test)

In [10]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [11]:
y_pred = clf.predict(X_test)
clf.score(X_test,y_test)

0.9396